In [1]:
import pandas as pd
import csv
import re
import hkvfewspy
import numpy as np
from datetime import datetime

## Data wrangling

In [12]:
# Create a dictionary with all the pathnames of the different instruments. We will only use the data file of the ALT instrument.
with open(r"D:\Projects\Pr\3916.10\Toeleveringen\Pathnames.csv", 'r') as f:
    reader = csv.reader(f)
    mydict = {}
    for row in reader:
        i, j = row
        mydict[i] = j

# Select the path of the file        
#path = mydict['STB']
path = r"D:\Projects\Pr\3916.10\Toeleveringen\STB_FL65_1902150941.dat"
# And get the name of the file: in future versions a script can be generated which handles all files
file = path.split('\\')[-1]

# From the file, search for the name of the instrument and the location
instrument = re.search('(.*)_FL', file).group(1)
location = re.search('{}_(.*)_'.format(instrument), file).group(1)

### METADATA: level n+NAP of 0 scale STB
locations = ['FL65', 'FL66', 'FL67', 'FL68', 'FL69', 'FL70'] 
STB_z0 = [-1.58, -1.51, -1.46, np.nan, -1.45, -1.65] # from file 190220 LakeSIDE Inmeting Meetpalen
z0 = dict(zip(locations, STB_z0)) 

### LOAD THE DATA AND CREATE A NICE DATAFRAME ###   

# Column names
colnames = ['TIMESTAMP','RECORD','EtroLevel']
# Create a dataframe of the data file (use the column names and skip the first 4 rows)        
df = pd.read_csv(path, names=colnames, header=None, skiprows = 4)

# DUMMY CODE
df['H_w_0'] = pd.to_numeric(df['EtroLevel'],errors='NAN')/100+z0[location]
# DUMMY CODE

# Subtract the year, month, day, time from the column 'TIME' and store in columns as integer
df['YEAR'] = pd.to_numeric(df['TIMESTAMP'].str.slice(0,4))
df['MONTH'] = pd.to_numeric(df['TIMESTAMP'].str.slice(5,7))
df['DAY'] = pd.to_numeric(df['TIMESTAMP'].str.slice(8,10))
df['HOUR'] = pd.to_numeric(df['TIMESTAMP'].str.slice(11,13))
df['MINUTE'] = pd.to_numeric(df['TIMESTAMP'].str.slice(14,16))
df['SECOND'] = pd.to_numeric(df['TIMESTAMP'].str.slice(17,22))

# Create a column with the datetime
df['DATETIME'] = pd.to_datetime(df.loc[:,['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'SECOND']])
# Use the datetime as the index of the dataframe
df.index = df.pop('DATETIME')
# Delete the columns 'TIME', 'YEAR', 'MONTH', etc
df.drop(['TIMESTAMP', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'SECOND','EtroLevel','RECORD'], axis=1, inplace=True)

# SET NAN to -9999
df.fillna(-9999, inplace=True)
df = df.head(17)

In [14]:
df.to_csv(r"D:\Projects\Pr\3916.10\Werkmap//csv_fractional_seconds.csv", index=True)

In [13]:
# Print DataFrame
df

,H_w_0
DATETIME,
2019-02-15 09:00:11.250,-1.68
2019-02-15 09:00:11.500,-1.63
2019-02-15 09:00:11.750,-1.73
2019-02-15 09:00:12.000,-1.78
2019-02-15 09:00:12.250,-1.78
2019-02-15 09:00:12.500,-1.78
2019-02-15 09:00:12.750,-1.73
2019-02-15 09:00:13.000,-1.73
2019-02-15 09:00:13.250,-1.73


In [15]:
# DATETIME index goes up to microseconds
df.index

DatetimeIndex(['2019-02-15 09:00:11.250000', '2019-02-15 09:00:11.500000',
               '2019-02-15 09:00:11.750000',        '2019-02-15 09:00:12',
               '2019-02-15 09:00:12.250000', '2019-02-15 09:00:12.500000',
               '2019-02-15 09:00:12.750000',        '2019-02-15 09:00:13',
               '2019-02-15 09:00:13.250000', '2019-02-15 09:00:13.500000',
               '2019-02-15 09:00:13.750000',        '2019-02-15 09:00:14',
               '2019-02-15 09:00:14.250000', '2019-02-15 09:00:14.500000',
               '2019-02-15 09:00:14.750000',        '2019-02-15 09:00:15',
               '2019-02-15 09:00:15.250000'],
              dtype='datetime64[ns]', name='DATETIME', freq=None)

## Create a FewsPi connection

In [16]:
pi = hkvfewspy.Pi() 
pi.setClient(wsdl='https://db.dmhoutribdijk.nl/FewsWebServices/fewspiservice?wsdl')

filters = pi.getFilters()
filter_id = filters['f_ruw_ongevalideerd_{}'.format(instrument)]['id']
locations = pi.getLocations(filterId=filter_id)
parameters = pi.getParameters()

# set variable settings
module_instance_id = 'ruw_ongevalideerd_STB'# 
location_id = locations.loc['locationId', location]
parameter_id = parameters.loc['id', 'H_w_0']

df_H_w_0 = pd.DataFrame(data = df['H_w_0'].values, index = df.index.values, columns = ['value'])

# create pi-timeseries object
pi_ts = pi.setPiTimeSeries()

# set a header object
pi_ts.write.header.moduleInstanceId(module_instance_id)
pi_ts.write.header.locationId(location_id)
pi_ts.write.header.parameterId(parameter_id)
pi_ts.write.header.missVal(-9999)

# set an events object (pandas.Series or pandas.DataFrame)
pi_ts.write.events(df_H_w_0)
pi_ts_xml = pi_ts.to.pi_xml()

In [19]:
df_H_w_0.head()

,value
2019-02-15 09:00:11.250,-1.68
2019-02-15 09:00:11.500,-1.63
2019-02-15 09:00:11.750,-1.73
2019-02-15 09:00:12.000,-1.78
2019-02-15 09:00:12.250,-1.78


In [18]:
filter_id, module_instance_id, location_id, parameter_id

('f_ruw_ongevalideerd.STB', 'ruw_ongevalideerd_STB', 'FL65', 'H.w.0')

In [ ]:
print(pi_ts.to.pi_xml()[0:2000])

In [ ]:
# http://fews.wldelft.nl/schemas/version1.0/pi-schemas/pi_timeseries.xsd
# ISO 8601 (hh:mm:ss.dsec e.g. 16:30:0.001)

In [ ]:
# Push data to FewsPi

In [ ]:
pi.putTimeSeriesForFilter(
   filterId=filter_id, 
   piTimeSeriesXmlContent=pi_ts.to.pi_xml()
)

In [ ]:
query = pi.setQueryParameters(prefill_defaults=True)

In [ ]:
query.parameterIds(['H.w.0'])
query.moduleInstanceIds(['ruw_ongevalideerd_STB'])
query.locationIds(['FL65'])
query.startTime(datetime(2019,2,15,9,0,11,250)) 
query.endTime(datetime(2019,2,15,9,10, 11,0))

In [ ]:
query.query

In [ ]:
df_get = pi.getTimeSeries(queryParameters=query, setFormat='df', header='longform' ,print_response=True)